In [250]:
import os
import os.path as osp
import pandas as pd
import numpy as np
import torch

In [285]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [251]:
def get_edges(x,y):
    src = np.random.randint(0,x,3500)
    dest = np.random.randint(0,y, 3500)
    return [src,dest]

In [252]:
from data import DailyData ,YearlyData

## Create graph structure

In [385]:
# Larget data
transform = T.Compose([T.ToUndirected(), T.AddSelfLoops(),T.NormalizeFeatures()])
root1 = osp.join(os.getcwd(), "dailyroot")
dailydata = DailyData(root1, transform = transform)

Processing...
Done!


In [387]:
dailydata[29]

HeteroData(
  node1={
    x=[125, 6],
    num_nodes=125
  },
  node2={
    x=[200, 6],
    num_nodes=200
  },
  node3={
    x=[10, 6],
    num_nodes=10
  },
  (node2, to, node1)={
    edge_index=[2, 350],
    edge_attr=[350, 6],
    edge_label=[350, 1]
  },
  (node3, to, node2)={
    edge_index=[2, 350],
    edge_attr=[350, 5],
    edge_label=[350, 1]
  },
  (node1, rev_to, node2)={
    edge_index=[2, 350],
    edge_attr=[350, 6],
    edge_label=[350, 1]
  },
  (node2, rev_to, node3)={
    edge_index=[2, 350],
    edge_attr=[350, 5],
    edge_label=[350, 1]
  }
)

In [255]:
#dataset = dailydata.shuffle()

In [390]:
from torch_geometric.loader import DataLoader
loader = DataLoader(dailydata, batch_size=4)
data = next(iter(loader))

In [393]:
data[("node2", "to", "node1")].edge_attr

tensor([[ 2.,  5.,  5.,  5.,  6.,  3.],
        [ 3.,  1.,  5.,  5.,  4., 10.],
        [ 4.,  4.,  8.,  3.,  3.,  4.],
        ...,
        [ 3.,  7.,  5.,  6.,  6.,  2.],
        [10.,  5.,  4.,  1.,  7., 10.],
        [ 1.,  3.,  7.,  9.,  5.,  8.]])

In [257]:
# InMemory data
root2 = osp.join(os.getcwd(), "yearlyroot")
yearlydata = YearlyData(root2)

In [258]:
yearlydata[0]

HeteroData(
  node1={
    x=[1250, 6],
    num_nodes=1250
  },
  node2={
    x=[2000, 6],
    num_nodes=2000
  },
  node3={
    x=[100, 6],
    num_nodes=100
  },
  (node2, to, node1)={
    edge_index=[2, 3500],
    edge_attr=[3500, 6],
    edge_label=[3500, 1]
  },
  (node3, to, node2)={
    edge_index=[2, 3500],
    edge_attr=[3500, 5],
    edge_label=[3500, 1]
  }
)

## Loading - Daily

In [259]:
n = (len(dailydata) + 9) // 10
train_dataset = dailydata[:2*n]
val_dataset = dailydata[2*n:2*n+n]
train_dataset = dailydata[2*n+n:]

In [260]:
train_dataset

DailyData(21)

In [370]:
data = T.ToUndirected()(dailydata)
data = T.AddSelfLoops()(data)
data = T.NormalizeFeatures()(data)

AttributeError: 'DailyData' object has no attribute 'edge_stores'

In [360]:
from torch_geometric.loader import DataLoader
loader = DataLoader(train_dataset, batch_size=4)

In [361]:
len(loader)

6

In [262]:
sampled_hetero_data = next(iter(loader))
sampled_hetero_data

HeteroDataBatch(
  node1={
    x=[500, 6],
    num_nodes=500,
    batch=[500],
    ptr=[5]
  },
  node2={
    x=[800, 6],
    num_nodes=800,
    batch=[800],
    ptr=[5]
  },
  node3={
    x=[40, 6],
    num_nodes=40,
    batch=[40],
    ptr=[5]
  },
  (node2, to, node1)={
    edge_index=[2, 1400],
    edge_attr=[1400, 6],
    edge_label=[1400, 1]
  },
  (node3, to, node2)={
    edge_index=[2, 1400],
    edge_attr=[1400, 5],
    edge_label=[1400, 1]
  }
)

### Transforms

In [378]:
loader = DataLoader(dailydata, batch_size=len(dailydata))
data = next(iter(loader))

In [ ]:
transform = T.Compose([T.ToUndirected(), T.AddSelfLoops()])
data = transform(data)
data

In [382]:
data = T.NormalizeFeatures()(dailydata)

AttributeError: 'DailyData' object has no attribute 'stores'

In [375]:
# Get full data stats for normalization
import torch_geometric.transforms as T
from collections import defaultdict
from copy import copy

def getfullstats(dailydata):
    """
    Pass the data into a loader to get the full stats
    Do this before Transforms that add reverse edges 
    Create a transform for this if you want to limit edge columns
    """
    loader = DataLoader(dailydata, batch_size=len(dailydata))
    data = next(iter(loader))
    edge_stats = defaultdict(dict)
    for edge in data.edge_types: 
        e = data[edge].edge_attr
        # get stats
        edge_stats[edge]["mean"] = e.mean(dim=0, keepdim=True)
        edge_stats[edge]["std"] = e.std(dim=0, keepdim=True)
        edge_stats[edge]["min"] = torch.min(e, 0).values
        edge_stats[edge]["max"] = torch.max(e, 0).values
    return edge_stats    

def edge_norm_fn(edge_stats):
    """
    Normalization of edge_attributes
    """
    for edge in data[0].edge_types:
        norm_data = copy(data)
        xmean = edge_stats[edge]["mean"]
        xstd = edge_stats[edge]["std"]
        xmin,xmax = edge_stats[edge]["min"],edge_stats[edge]["max"]
        for i in range(len(data)): 
            x = norm_data[i][edge].edge_attr
            del data[i][edge].edge_attr
            norm_data[i][edge].edge_attr = (x-xmean)/(xmax-xmin)
    return norm_data


edge_stats = getfullstats(dailydata)

In [377]:
new_data = edge_norm_fn(dailydata)
data[3][dailydata[0].edge_types[1]].edge_attr
new_data[3][dailydata[0].edge_types[1]].edge_attr

tensor([[ 5.,  3.,  6.,  7.,  1.],
        [ 9.,  8.,  0.,  8.,  4.],
        [ 2.,  5.,  7., 10.,  1.],
        ...,
        [10.,  7.,  7.,  3.,  5.],
        [ 2.,  2.,  9.,  7.,  1.],
        [ 3.,  8.,  3.,  8.,  7.]])

tensor([[ 5.,  3.,  6.,  7.,  1.],
        [ 9.,  8.,  0.,  8.,  4.],
        [ 2.,  5.,  7., 10.,  1.],
        ...,
        [10.,  7.,  7.,  3.,  5.],
        [ 2.,  2.,  9.,  7.,  1.],
        [ 3.,  8.,  3.,  8.,  7.]])

In [342]:
data = dailydata[0]
edge = dailydata[0].edge_types[0]
x = data[edge].edge_attr
xmean = edge_stats[edge]["mean"]
xmin,xmax = edge_stats[edge]["min"],edge_stats[edge]["max"]

x
data[edge].edge_attr = (x-xmean)/(xmax-xmin)
data[edge].edge_attr

tensor([[ 8.,  4.,  9.,  5.,  5.,  3.],
        [ 4.,  1.,  2.,  8.,  7.,  8.],
        [ 7.,  7.,  4.,  7.,  8.,  5.],
        ...,
        [ 1.,  4.,  1.,  2.,  9.,  5.],
        [ 8.,  2.,  5.,  8., 10.,  7.],
        [ 9.,  8.,  1.,  1.,  8.,  2.]])

tensor([[ 0.3030, -0.1022,  0.4018,  0.0022,  0.0026, -0.2051],
        [-0.0970, -0.4022, -0.2982,  0.3022,  0.2026,  0.2949],
        [ 0.2030,  0.1978, -0.0982,  0.2022,  0.3026, -0.0051],
        ...,
        [-0.3970, -0.1022, -0.3982, -0.2978,  0.4026, -0.0051],
        [ 0.3030, -0.3022,  0.0018,  0.3022,  0.5026,  0.1949],
        [ 0.4030,  0.2978, -0.3982, -0.3978,  0.3026, -0.3051]])

In [345]:
dailydata[0][edge].edge_attr

tensor([[ 8.,  4.,  9.,  5.,  5.,  3.],
        [ 4.,  1.,  2.,  8.,  7.,  8.],
        [ 7.,  7.,  4.,  7.,  8.,  5.],
        ...,
        [ 1.,  4.,  1.,  2.,  9.,  5.],
        [ 8.,  2.,  5.,  8., 10.,  7.],
        [ 9.,  8.,  1.,  1.,  8.,  2.]])

In [ ]:
data[i][edge].edge_attr = (x-xmean)/(xmax-xmin)

In [335]:
transform(dailydata[0])

In [336]:
data_t[0][("node2", "to", "node1")].edge_attr

tensor([[ 8.,  4.,  9.,  5.,  5.,  3.],
        [ 4.,  1.,  2.,  8.,  7.,  8.],
        [ 7.,  7.,  4.,  7.,  8.,  5.],
        ...,
        [ 1.,  4.,  1.,  2.,  9.,  5.],
        [ 8.,  2.,  5.,  8., 10.,  7.],
        [ 9.,  8.,  1.,  1.,  8.,  2.]])

In [ ]:
def transform_fn(data):
   data[edge_type].edge_attr
   return data

In [ ]:
# test Data split and creation

## Changes

In [ ]:
def train(model, optimizer, train_loader, device):
    model.train()

    for data in train_loader:
        optimizer.zero_grad()
        data = data.to(device)
        out = model(data.pos, data.batch)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()



val_losses, accs, durations = [], [], []

if torch.cuda.is_available():
        torch.cuda.synchronize()
    t_start = time.perf_counter()

for epoch in range(epochs):
    train_loss = train(model, optimizer, train_loader)
    val_losses.append(eval_loss(model, val_loader))
    accs.append(eval_acc(model, test_loader))
    eval_info = {
        'fold': fold,
        'epoch': epoch,
        'train_loss': train_loss,
        'val_loss': val_losses[-1],
        'test_acc': accs[-1],
    }
    
if torch.cuda.is_available():
    torch.cuda.synchronize()
t_end = time.perf_counter()
durations.append(t_end - t_start)

print(f'Epoch: {epoch:03d}, Test: {test_acc:.4f}, '
              f'Duration: {t_end - t_start:.2f}')

